# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df=pd.read_csv('../WeatherPy/city_weather_info.csv')
vacation_df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Temperature,Humidity,Cloud,Wind
0,0,New Norfolk,AU,-42.7826,147.0587,52.99,53,100,3.00
1,1,Cherskiy,RU,68.7500,161.3000,24.06,79,100,9.31
2,2,Mar del Plata,AR,-38.0023,-57.5575,59.56,91,90,5.01
3,3,Saint-Philippe,RE,-21.3585,55.7679,82.26,71,16,10.22
4,4,Valverde del Camino,ES,37.5751,-6.7543,46.89,53,94,4.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = vacation_df[["Latitude", "Longitude"]].astype(float)
humidity = vacation_df['Humidity'].astype(float)
maxhumidity = humidity.max()

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations,
                                 weights=humidity, 
                                 dissipating=False, 
                                 max_intensity=maxhumidity,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
ideal_weather_df = vacation_df.loc[(vacation_df['Temperature']>70)&(vacation_df['Temperature']<80)&(vacation_df['Wind']<10)&(vacation_df['Cloud']==0)]

ideal_weather_df = ideal_weather_df.reset_index(drop=True)

ideal_weather_df = ideal_weather_df.dropna()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df=pd.DataFrame(ideal_weather_df, columns=['City','Country','Latitude','Longitude', 'Temperature', 'Humidity','Cloud','Wind'])

hotel_df['Hotel Name']=""
hotel_df

,City,Country,Latitude,Longitude,Temperature,Humidity,Cloud,Wind,Hotel Name
0,Najrān,SA,17.4924,44.1277,75.85,23,0,2.48,
1,Ateli Mandi,IN,28.1000,76.2833,79.52,16,0,5.19,
2,Sardhana,IN,29.1500,77.6167,78.13,21,0,1.16,
3,Alofi,NU,-19.0595,-169.9187,71.49,83,0,5.75,
4,Pāsighāt,IN,28.0667,95.3333,78.85,40,0,3.62,
5,Kohlu,PK,29.8962,69.2520,70.16,9,0,7.34,


In [6]:
hotels = []

for i in range(len(hotel_df)):
    LAT=hotel_df.iloc[i]['Latitude']
    LONG=hotel_df.iloc[i]['Longitude']
    
    params = {
        "location":f'{LAT},{LONG}',
        "radius":5000,
        "types":"hotel",
        "key":g_key
    }
    
    url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(url,params=params).json()
    
    try:
        hotels.append(response['results'][0]['name'])
    except:
        hotels.append("")
        
hotel_df["Hotel Name"] = hotels
hotel_df

,City,Country,Latitude,Longitude,Temperature,Humidity,Cloud,Wind,Hotel Name
0,Najrān,SA,17.4924,44.1277,75.85,23,0,2.48,Najran
1,Ateli Mandi,IN,28.1000,76.2833,79.52,16,0,5.19,Ateli Mandi
2,Sardhana,IN,29.1500,77.6167,78.13,21,0,1.16,Sardhana
3,Alofi,NU,-19.0595,-169.9187,71.49,83,0,5.75,Alofi
4,Pāsighāt,IN,28.0667,95.3333,78.85,40,0,3.62,Pasighat
5,Kohlu,PK,29.8962,69.2520,70.16,9,0,7.34,Kohlu


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [8]:
# Create Marker Layer
locations = hotel_df[["Latitude", "Longitude"]]
hotel_layer = gmaps.marker_layer(
     locations,                                                       
     info_box_content= hotel_info
)

# fill_color='red',
#      stroke_color='rgba(0, 0, 150, 0.4)', scale=8, 
fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))